In [ ]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorflow as tf

%matplotlib inline
warnings.filterwarnings("ignore")

# Set seeds to make the experiment more reproducible.
# import tensorflow as tf
# tf.random.set_seed()
# from numpy.random import seed
# seed(1)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
train = pd.read_csv('train_time_data_new.csv')
test = pd.read_csv('test_time_data_new.csv')
# train, test

In [ ]:
train.drop(columns=['Stop_Time'], inplace=True)
test.drop(columns=['Stop_Time'], inplace=True)

In [ ]:
l1 = ['Time_Since_Shift_Start','Time_Since_Morning','Station_Name_1-Stop_Reason_1-Remark_1',
 'Station_Name_1-Stop_Reason_3-Remark_12',
 'Station_Name_1-Stop_Reason_3-Remark_8',
 'Station_Name_10-Stop_Reason_4-Remark_4',
 'Station_Name_2-Stop_Reason_2-Remark_2',
 'Station_Name_3-Stop_Reason_3-Remark_10',
 'Station_Name_3-Stop_Reason_3-Remark_3',
 'Station_Name_3-Stop_Reason_3-Remark_6',
 'Station_Name_3-Stop_Reason_3-Remark_7',
 'Station_Name_3-Stop_Reason_4-Remark_4',
 'Station_Name_4-Stop_Reason_3-Remark_5',
 'Station_Name_4-Stop_Reason_4-Remark_4',
 'Station_Name_5-Stop_Reason_4-Remark_4',
 'Station_Name_6-Stop_Reason_3-Remark_13',
 'Station_Name_6-Stop_Reason_3-Remark_15',
 'Station_Name_6-Stop_Reason_4-Remark_4',
 'Station_Name_7-Stop_Reason_3-Remark_14',
 'Station_Name_7-Stop_Reason_3-Remark_9',
 'Station_Name_7-Stop_Reason_4-Remark_4',
 'Station_Name_8-Stop_Reason_3-Remark_11',
 'Station_Name_8-Stop_Reason_4-Remark_4',
 'Station_Name_9-Stop_Reason_4-Remark_4']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
cpu_list = [i for i in train.columns if i != 'Stop_Time']
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(train[cpu_list])
scaled_test = scaler.transform(test[cpu_list])

In [ ]:
def split_sequence(sequence, look_back, forecast_horizon):
 X, y = list(), list()
 for i in range(len(sequence)):
   lag_end = i + look_back
   forecast_end = lag_end + forecast_horizon
   if forecast_end > len(sequence):
     break
   seq_x, seq_y = sequence[i:lag_end], sequence[lag_end:forecast_end]
   X.append(seq_x)
   y.append(seq_y)
 return np.array(X), np.array(y)

In [ ]:
n_features = len(cpu_list)
X_train, y_train = split_sequence(scaled_train, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
X_test, y_test = split_sequence(scaled_test, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5331, 24, 30)
(5331, 4, 30)
(2758, 24, 30)
(2758, 4, 30)


In [ ]:
checkpoint_filepath = 'path_to_checkpoint_filepath'
checkpoint_callback = ModelCheckpoint(
 filepath=checkpoint_filepath,
 save_weights_only=False,
 monitor='val_loss',
 mode='min',
 save_best_only=True)
early_stopping_callback = EarlyStopping(
 monitor="val_loss",
 min_delta=0.005,
 patience=10,
 mode="min"
)
rlrop_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, mode='min', patience=3, min_lr=0.001)

In [ ]:
def inverse_transform(y_test, yhat):
 y_test_reshaped = y_test.reshape(-1, y_test.shape[-1])
 yhat_reshaped = yhat.reshape(-1, yhat.shape[-1])
 yhat_inverse = scaler.inverse_transform(yhat_reshaped)
 y_test_inverse = scaler.inverse_transform(y_test_reshaped)
 return yhat_inverse, y_test_inverse

In [ ]:
def evaluate_forecast(y_test_inverse, yhat_inverse):
 mse_ = tf.keras.losses.MeanSquaredError()
 mae_ = tf.keras.losses.MeanAbsoluteError()
 mape_ = tf.keras.losses.MeanAbsolutePercentageError()
 mae = mae_(y_test_inverse,yhat_inverse)
 print('mae:', mae)
 mse = mse_(y_test_inverse,yhat_inverse)
 print('mse:', mse)
 mape = mape_(y_test_inverse,yhat_inverse)
 print('mape:', mape)

In [ ]:
epochs = 50
batch_size = 32
validation = 0.1

In [ ]:
num_samples = 6624

In [ ]:
evaluate_forecast(y_test_inverse, yhat_inverse)

mae: tf.Tensor(0.23641124, shape=(), dtype=float32)
mse: tf.Tensor(0.61856437, shape=(), dtype=float32)
mape: tf.Tensor(113829970.0, shape=(), dtype=float32)


In [ ]:
test2 = pd.read_csv('test_data_21onw.csv')
test2.drop(columns=['Stop_Time'],inplace=True)
test2.head()

,Time_Since_Morning,Station_Name-Station_Name_1,Station_Name-Station_Name_2,Station_Name-Station_Name_3,Station_Name-Station_Name_4,Station_Name-Station_Name_5,Station_Name-Station_Name_6,Station_Name-Station_Name_7,Station_Name-Station_Name_8,Station_Name-Station_Name_9,...,Remarks-Remark_15,Remarks-Remark_2,Remarks-Remark_3,Remarks-Remark_4,Remarks-Remark_5,Remarks-Remark_6,Remarks-Remark_7,Remarks-Remark_8,Remarks-Remark_9,Time_Since_Shift_Start
0,1.0,11,6,4,2,4,4,1,2,0,...,0,6,15,1,1,0,0,0,0,1.0
1,2.0,15,6,5,8,4,3,0,8,1,...,0,6,23,0,6,0,0,0,0,2.0
2,3.0,10,6,0,2,2,1,0,2,4,...,0,6,11,0,0,0,0,0,0,3.0
3,4.0,7,6,0,2,0,1,0,0,4,...,0,6,6,0,1,0,0,0,0,4.0
4,5.0,6,6,1,3,2,4,1,2,3,...,0,6,11,3,0,2,0,0,0,5.0


In [ ]:
LOOK_BACK = 6
FORECAST_RANGE = 1

In [ ]:
n_features = len(cpu_list)
X_train, y_train = split_sequence(scaled_train, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
X_test, y_test = split_sequence(scaled_test, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5352, 6, 30)
(5352, 1, 30)
(2779, 6, 30)
(2779, 1, 30)


In [ ]:
#create model with convolutional layer and pooling layer and lstm layer
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
conv_layer = Conv1D(filters=64, kernel_size=2, activation='relu')(input_layer)
pool_layer = MaxPooling1D(pool_size=2)(conv_layer)
lstm_layer = LSTM(50, activation='tanh')(pool_layer)
repeat_layer = RepeatVector(FORECAST_RANGE)(lstm_layer)
lstm_layer_2 = LSTM(50, activation='tanh', return_sequences=True)(repeat_layer)
dropout_layer = Dropout(0.2)(lstm_layer_2)
output_layer = Dense(n_features, activation='linear')(dropout_layer)
new_model = Model(inputs=input_layer, outputs=output_layer)
new_model.compile(optimizer='adam', loss='mse', metrics=['mse'])


In [ ]:
    #plot model
plot_model(model=new_model, show_shapes=True)
history = new_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback,checkpoint_callback, rlrop_callback])

yhat = new_model.predict(X_test, verbose=0)
yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)
evaluate_forecast(y_test_inverse, yhat_inverse)

Epoch 1/50
151/151 [==============================] - ETA: 0s - loss: 0.0187 - mse: 0.0187

INFO:tensorflow:Assets written to: path_to_checkpoint_filepath\assets


INFO:tensorflow:Assets written to: path_to_checkpoint_filepath\assets


151/151 [==============================] - 18s 58ms/step - loss: 0.0187 - mse: 0.0187 - val_loss: 0.0108 - val_mse: 0.0108 - lr: 0.0010
Epoch 2/50
147/151 [============================>.] - ETA: 0s - loss: 0.0098 - mse: 0.0098

INFO:tensorflow:Assets written to: path_to_checkpoint_filepath\assets


...

INFO:tensorflow:Assets written to: path_to_checkpoint_filepath\assets


151/151 [==============================] - 7s 50ms/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.0049 - val_mse: 0.0049 - lr: 0.0010
Epoch 14/50
151/151 [==============================] - 1s 9ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.0049 - val_mse: 0.0049 - lr: 0.0010
Epoch 15/50
147/151 [============================>.] - ETA: 0s - loss: 0.0037 - mse: 0.0037

INFO:tensorflow:Assets written to: path_to_checkpoint_filepath\assets


INFO:tensorflow:Assets written to: path_to_checkpoint_filepath\assets


151/151 [==============================] - 8s 50ms/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.0047 - val_mse: 0.0047 - lr: 0.0010
mae: tf.Tensor(0.472809, shape=(), dtype=float32)
mse: tf.Tensor(1.9031357, shape=(), dtype=float32)
mape: tf.Tensor(160639620.0, shape=(), dtype=float32)


In [ ]:
new_model.save('new_model_early_stop.h5')


In [ ]:
new_model_stop = tf.keras.models.load_model('new_model_early_stop.h5')
print(new_model_stop.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6, 30)]           0         
                                                                 
 conv1d (Conv1D)             (None, 5, 64)             3904      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2, 64)            0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 50)                23000     
                                                                 
 repeat_vector (RepeatVector  (None, 1, 50)            0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 1, 50)             20200 